# This notebook processes raw FAC data
FAC data from 2014-2017 was downloaded in bulk from https://harvester.census.gov/facdissem/PublicDataDownloads.aspx

## Process findings.txt

In [2]:
import numpy as np
import pandas as pd
import re

fac_path = '/data/ihwang/data_journalism/allfac/'
findings = 'findings_1000.txt'
###############################################################################


data = []
with open(fac_path + findings, 'r') as f:
    for line in f:
        if line[0] != ' ':
            header = line.split(',')
        else:
            fields = re.sub('\s', '', line).split(',')
            [DBKEY,AUDITYEAR,ELECAUDITSID,
             ELECAUDITFINDINGSID] = [int(field) for field in fields[:4]]
            # Can be multiple FINDINGSREFNUMS which makes it necessary to parse
            # file line-by-line (instead of using pandas.csv_read)
            [*FINDINGSREFNUMS] = fields[4:-9]
            [TYPEREQUIREMENT, MODIFIEDOPINION,OTHERNONCOMPLIANCE,
             MATERIALWEAKNESS,SIGNIFICANTDEFICIENCY,OTHERFINDINGS,QCOSTS,
             REPEATFINDING,PRIORFINDINGREFNUMS] = fields[-9:]
            data.append(
                [
                    DBKEY,AUDITYEAR,ELECAUDITSID,ELECAUDITFINDINGSID,
                    FINDINGSREFNUMS, TYPEREQUIREMENT,MODIFIEDOPINION,
                    OTHERNONCOMPLIANCE,MATERIALWEAKNESS,
                    SIGNIFICANTDEFICIENCY,OTHERFINDINGS,QCOSTS,
                    REPEATFINDING,PRIORFINDINGREFNUMS
                ]
            )
findings_all = pd.DataFrame(data, columns=header)
findings_df = findings_all[
    [
        'DBKEY', 'AUDITYEAR', 'TYPEREQUIREMENT', 'MODIFIEDOPINION', 
        'OTHERNONCOMPLIANCE','MATERIALWEAKNESS', 'SIGNIFICANTDEFICIENCY',
        'OTHERFINDINGS', 'QCOSTS'
    ]
]
findings_df.head()
# findings_df.to_pickle(fac_path + 'processed/findings.pkl')

,DBKEY,AUDITYEAR,TYPEREQUIREMENT,MODIFIEDOPINION,OTHERNONCOMPLIANCE,MATERIALWEAKNESS,SIGNIFICANTDEFICIENCY,OTHERFINDINGS,QCOSTS
0,193062,2013,M,N,Y,N,Y,N,N
1,193062,2013,M,N,Y,N,Y,N,N
2,193062,2013,M,N,Y,N,Y,N,N
3,193062,2013,M,N,Y,N,Y,N,N
4,193062,2013,B,N,Y,N,Y,N,N


In [744]:
findings_all.columns

Index(['DBKEY', 'AUDITYEAR', 'ELECAUDITSID', 'ELECAUDITFINDINGSID',
       'FINDINGSREFNUMS', 'TYPEREQUIREMENT', 'MODIFIEDOPINION',
       'OTHERNONCOMPLIANCE', 'MATERIALWEAKNESS', 'SIGNIFICANTDEFICIENCY',
       'OTHERFINDINGS', 'QCOSTS', 'REPEATFINDING',
       'PRIORFINDINGREFNUMS                                                                                                                                                                                                         \n'],
      dtype='object')

In [743]:
findings_all.loc[986:990]

,DBKEY,AUDITYEAR,ELECAUDITSID,ELECAUDITFINDINGSID,FINDINGSREFNUMS,TYPEREQUIREMENT,MODIFIEDOPINION,OTHERNONCOMPLIANCE,MATERIALWEAKNESS,SIGNIFICANTDEFICIENCY,OTHERFINDINGS,QCOSTS,REPEATFINDING,PRIORFINDINGREFNUMS
986,45544,2013,14017847,435,[2013-01],F,N,N,N,Y,N,N,,
987,45468,2013,14085228,6686,[2013-001],N,N,N,N,Y,N,N,,
988,33432,2013,14112119,8168,[2013-01],M,N,Y,N,N,N,N,,
989,95224,2013,14133978,10060,[2013-1],P,N,N,N,Y,N,N,,
990,220319,2013,14037704,3390,[1],N,N,Y,N,N,N,N,,


## Process general.txt

In [57]:
fac_path = '/data/ihwang/data_journalism/allfac/'
general = 'general.txt'
###############################################################################


general_all = pd.read_csv(
    fac_path + general, encoding = 'ISO-8859-1', low_memory=False
)  # low memory parameter for mixed types
general_df = general_all[
    [
        'DBKEY', 'AUDITYEAR', 'STATE', 'CPASTATE',
        'TYPEOFENTITY', 'OVERSIGHTAGENCY', 'TOTFEDEXPEND'
    ]
]
general_df = general_df.assign(
    AUDITEENAME=general_all['AUDITEENAME'].str.strip().str.replace(r'[^\w\s]', '')
)
general_df = general_df.assign(
    CPAFIRMNAME=general_all['CPAFIRMNAME'].str.strip().str.replace(r'[^\w\s]', '')
)
general_df = general_df.assign(
    ZIPCODE=pd.to_numeric(general_all['ZIPCODE'].str[:5], errors='coerce')
)
general_df = general_df.assign(
    CPAZIPCODE=pd.to_numeric(general_all['CPAZIPCODE'].str[:5], errors='coerce')
)
general_df = general_df.assign(
    TYPEREPORT_FS=general_all['TYPEREPORT_FS'].str.strip()
)
general_df = general_df.assign(
    TYPEREPORT_MP=general_all['TYPEREPORT_MP'].str.strip()
)
general_df = general_df.assign(
    CITY=general_all['CITY'].str.strip()
)
general_df = general_df.assign(
    CPACITY=general_all['CPACITY'].str.strip()
)
general_df = general_df.assign(
    EIN=pd.to_numeric(general_all['EIN'].str.split('-').str.get(-1))
)
general_df.head()
general_df.to_pickle(fac_path + 'processed/general.pkl')

In [56]:
print(general_all.columns)
general_all.head()

Index(['AUDITYEAR', 'DBKEY', 'TYPEOFENTITY', 'FYENDDATE', 'AUDITTYPE',
       'PERIODCOVERED', 'NUMBERMONTHS', 'EIN', 'MULTIPLEEINS', 'EINSUBCODE',
       'DUNS', 'MULTIPLEDUNS', 'AUDITEENAME', 'STREET1', 'STREET2', 'CITY',
       'STATE', 'ZIPCODE', 'AUDITEECONTACT', 'AUDITEETITLE', 'AUDITEEPHONE',
       'AUDITEEFAX', 'AUDITEEEMAIL', 'AUDITEEDATESIGNED', 'AUDITEENAMETITLE',
       'CPAFIRMNAME', 'CPASTREET1', 'CPASTREET2', 'CPACITY', 'CPASTATE',
       'CPAZIPCODE', 'CPACONTACT', 'CPATITLE', 'CPAPHONE', 'CPAFAX',
       'CPAEMAIL', 'CPADATESIGNED', 'COG_OVER', 'COGAGENCY', 'OVERSIGHTAGENCY',
       'TYPEREPORT_FS', 'SP_FRAMEWORK', 'SP_FRAMEWORK_REQUIRED',
       'TYPEREPORT_SP_FRAMEWORK', 'GOINGCONCERN', 'REPORTABLECONDITION',
       'MATERIALWEAKNESS', 'MATERIALNONCOMPLIANCE', 'TYPEREPORT_MP',
       'DUP_REPORTS', 'DOLLARTHRESHOLD', 'LOWRISK', 'REPORTABLECONDITION_MP',
       'MATERIALWEAKNESS_MP', 'QCOSTS', 'CYFINDINGS', 'PYSCHEDULE',
       'TOTFEDEXPEND', 'DATEFIREWALL', 'PREVIO

,AUDITYEAR,DBKEY,TYPEOFENTITY,FYENDDATE,AUDITTYPE,PERIODCOVERED,NUMBERMONTHS,EIN,MULTIPLEEINS,EINSUBCODE,...,QCOSTS,CYFINDINGS,PYSCHEDULE,TOTFEDEXPEND,DATEFIREWALL,PREVIOUSDATEFIREWALL,REPORTREQUIRED,MULTIPLE_CPAS,AUDITOR_EIN,FACACCEPTEDDATE
0,1997,100000,,30-SEP-97,S,A,,730776899,N,000,...,N,,,2195498,15-MAY-98,,Y,,,05-MAR-98 ...
1,1998,100000,,30-SEP-98,S,A,,730776899,N,000,...,N,,,2443678,17-JUN-99,,Y,,,24-MAR-99 ...
2,1999,100000,,30-SEP-99,S,A,,730776899,N,000,...,N,,,2715892,15-FEB-00,,N,,,03-FEB-00 ...
3,2000,100000,,30-SEP-00,S,A,,730776899,N,000,...,N,,,2875293,28-FEB-01,,N,,,19-JAN-01 ...
4,2001,100000,908,30-SEP-01,S,A,,730776899,N,000,...,N,N,N,2875169,12-MAR-02,,,,,17-JAN-02 ...


## Process agency prefix table
Raw data copied and pasted from https://harvester.census.gov/facdissem/Documents/PublicUserManual.pdf

In [71]:
import pickle


fac_path = '/data/ihwang/data_journalism/allfac/'
prefix_table = 'prefix_table.txt'


agency_dict = {}
with open(fac_path + prefix_table, 'r') as f:
    for line in f:
        data = line.strip().split(' ')
        prefix = data[0]
        agency = data[1:-1]
        agency_dict[prefix] = ' '.join(agency)
        
        
with open(fac_path + '/processed/agency_dict', 'wb') as f_w:
    pickle.dump(agency_dict, f_w)

## Process entity codes table
Raw data copied and pasted from https://harvester.census.gov/facdissem/Documents/PublicUserManual.pdf

In [72]:
fac_path = '/data/ihwang/data_journalism/allfac/'
entity_table = 'entity_table.txt'


entity_dict = {}
with open(fac_path + entity_table, 'r') as f:
    for line in f:
        data = line.strip().split(' ')
        entity = data[:-1]
        code = data[-1]
        entity_dict[code] = ' '.join(entity)
        
        
with open(fac_path + '/processed/entity_dict', 'wb') as f_w:
    pickle.dump(entity_dict, f_w)